In [1]:
import pandas as pd
import pycountry
import ast
import numpy as np
import rasterio
from rasterio.warp import calculate_default_transform, reproject, Resampling
import duckdb
from tqdm import tqdm

In [5]:
# Funções auxiliares
def construir_mapeamento_subdivisoes():
    mapping = {}
    for subdiv in pycountry.subdivisions:
	    code = subdiv.code.split('-')[-1].upper()
        country_code = subdiv.country_code
        mapping.setdefault(code, set()).add(country_code)
    return mapping

IndentationError: unindent does not match any outer indentation level (<string>, line 6)

In [6]:
def inferir_pais(linha, mapeamento_subdivisoes):
    sighting_state = linha.get('sightingState', None)
    if pd.isna(sighting_state):
        return None
    sighting_state = sighting_state.upper().strip()
    possible_countries = mapeamento_subdivisoes.get(sighting_state, set())
    if not possible_countries:
        return None
    elif len(possible_countries) == 1:
        return possible_countries.pop()
    else:
        return "Ambíguo"

In [7]:
def hex_to_rgb(hex_color):
    return tuple(int(hex_color[i:i+2], 16) for i in (0, 2, 4))

In [8]:
# Classe para manipulação geológica
class GeologicalMapper:
    def __init__(self, mapa_path, geological_df):
        self.mapa_path = mapa_path
        self.geological_df = geological_df.copy()
        self.geological_df['RGB_array'] = self.geological_df['RGB'].apply(np.array)
        self.src = self._abrir_mapa()
        
    def _abrir_mapa(self):
        src = rasterio.open(self.mapa_path)
        if src.crs.to_string() != 'EPSG:4326':
            src = self._reprojetar_raster(src)
        return src
    
    def _reprojetar_raster(self, src):
        dst_crs = 'EPSG:4326'
        transform, width, height = calculate_default_transform(
            src.crs, dst_crs, src.width, src.height, *src.bounds)
        kwargs = src.meta.copy()
        kwargs.update({'crs': dst_crs, 'transform': transform, 'width': width, 'height': height})
        
        memfile = rasterio.io.MemoryFile()
        with memfile.open(**kwargs) as dst:
            for i in range(1, src.count + 1):
                reproject(
                    source=rasterio.band(src, i),
                    destination=rasterio.band(dst, i),
                    src_transform=src.transform,
                    src_crs=src.crs,
                    dst_transform=transform,
                    dst_crs=dst_crs,
                    resampling=Resampling.nearest)
            return memfile.open()
    
    def obter_cor(self, latitude, longitude):
        try:
            px, py = self.src.index(longitude, latitude)
            if (px < 0 or px >= self.src.width) or (py < 0 or py >= self.src.height):
                return None
            bandas = self.src.read([1, 2, 3])
            R, G, B = bandas[:, py, px]
            return (R, G, B)
        except ValueError:
            return None
    
    def obter_caracteristica_geologica(self, latitude, longitude):
        if pd.isna(latitude) or pd.isna(longitude):
            return "Coordenadas não disponíveis"
        rgb = self.obter_cor(latitude, longitude)
        if rgb is None:
            return "Localização fora do mapa"
        rgb_point = np.array(rgb)
        distances = self.geological_df['RGB_array'].apply(lambda x: np.linalg.norm(rgb_point - x))
        min_distance_idx = distances.idxmin()
        caracteristica = self.geological_df.loc[min_distance_idx, 'Tipo de Rocha']
        return caracteristica

In [9]:
# Função principal para responder perguntas
def responder_perguntas(
	linha,
	mapeamento,
	perguntas_irrelevantes_para_criancas,
	mapeamento_subdivisoes,
	mapeamento_caracteristica_para_perguntas,
	perguntas_geologicas
):
	respostas = {}
	
	# Processar perguntas geológicas
	caracteristica_geologica = linha.get('Caracteristica Geologica', None)
	for pergunta in perguntas_geologicas:
		respostas[pergunta] = "Não"
	if caracteristica_geologica:
		respostas_caracteristica = mapeamento_caracteristica_para_perguntas.get(caracteristica_geologica, {})
		for pergunta, resposta in respostas_caracteristica.items():
			respostas[pergunta] = resposta
	else:
		for pergunta in perguntas_geologicas:
			respostas[pergunta] = "Não Sei"
	respostas["Características Geológicas Inferidas"] = caracteristica_geologica or "Não sei"
	
	# Determinar se é criança
	is_child = False
	if pd.notna(linha['computedMissingMaxAge']):
		is_child = linha['computedMissingMaxAge'] < 18
	
	# Inferir país
	country_code = inferir_pais(linha, mapeamento_subdivisoes)
	if country_code == 'BR':
		is_brazilian = "Sim"
	elif country_code and country_code != 'BR' and country_code != "Não sei":
		is_brazilian = "Não"
	else:
		is_brazilian = "Não sei"
	
	for pergunta in perguntas:
		if is_child and pergunta in perguntas_irrelevantes_para_criancas:
			respostas[pergunta] = "Não"
			continue
		
		colunas = mapeamento.get(pergunta, [])
		
		if pergunta == "Nome?":
			nomes = [str(linha[col]).strip() for col in colunas if pd.notna(linha[col])]
			respostas[pergunta] = ' '.join(nomes) if nomes else None
		elif pergunta == "Sexo?":
			respostas[pergunta] = linha.get(colunas[0], "Não sei") if colunas else "Não sei"
		elif pergunta == "O desaparecido é homem?":
			sexo = linha.get(colunas[0], None) if colunas else None
			if pd.notna(sexo):
				respostas[pergunta] = "Sim" if sexo.lower() == "male" else "Não"
			else:
				respostas[pergunta] = "Não sei"
		elif pergunta == "Qual data do desaparecimento?":
			data = linha.get(colunas[0], None) if colunas else None
			if pd.notna(data):
				data_parsed = pd.to_datetime(data, errors='coerce')
				respostas[pergunta] = data_parsed.strftime('%Y-%m-%d') if pd.notna(data_parsed) else None
			else:
				respostas[pergunta] = None
		elif pergunta == "O desaparecido tem olhos claros?":
			cores = [str(linha[col]).strip().lower() for col in colunas if pd.notna(linha[col])]
			olhos_claros = any(cor in ["blue", "green"] for cor in cores)
			respostas[pergunta] = "Sim" if olhos_claros else "Não"
		elif "características físicas" in pergunta:
			etnia = linha.get(colunas[0], "") if colunas else ""
			if pd.notna(etnia):
				if "asiáticas" in pergunta:
					respostas[pergunta] = "Sim" if "Asian" in etnia else "Não"
				elif "africanas" in pergunta:
					respostas[pergunta] = "Sim" if "Black / African American" in etnia else "Não"
				elif "europeias" in pergunta:
					respostas[pergunta] = "Sim" if "White / Caucasian" in etnia else "Não"
				else:
					respostas[pergunta] = "Não"
			else:
				respostas[pergunta] = "Não"
		elif pergunta == "O desaparecido está morto?":
			resolvido = linha.get(colunas[0], None) if colunas else None
			if pd.notna(resolvido):
				respostas[pergunta] = "Sim" if resolvido else "Não"
			else:
				respostas[pergunta] = "Não sei"
		elif pergunta == "O desaparecido costumava usar carro?":
			vehicles = linha.get('vehicles', '')
			if pd.isna(vehicles) or vehicles in ['[]', '']:
				respostas[pergunta] = "Não"
			else:
				try:
					vehicles_list = ast.literal_eval(vehicles)
					respostas[pergunta] = "Sim" if vehicles_list else "Não"
				except:
					respostas[pergunta] = "Não sei"
		elif pergunta in ["O desaparecido costumava usar ônibus?", "O desaparecido costumava usar aplicativo de transporte?"]:
			respostas[pergunta] = "Não sei"
		elif pergunta == "O desaparecido é Brasileiro?":
			respostas[pergunta] = is_brazilian
		elif pergunta.startswith("O desaparecido iria para"):
			respostas[pergunta] = "Não sei"
		elif pergunta in ["O desaparecido estava procurando emprego?", "O desaparecido é desempregado?"]:
			respostas[pergunta] = "Não" if is_child else "Não sei"
		elif pergunta == "O desaparecido já havia comentado sobre a possibilidade de deixar trabalho?":
			respostas[pergunta] = "Não sei"
		else:
			respostas[pergunta] = "Não sei"
	return respostas

In [10]:
# Aplicar funções e gerar respostas
def gerar_respostas(df):
    respostas_df = df.apply(
        lambda row: responder_perguntas(
            row,
            mapeamento,
            perguntas_irrelevantes_para_criancas,
            mapeamento_subdivisoes,
            mapeamento_caracteristica_para_perguntas,
            perguntas_geologicas
        ),
        axis=1
    )
    respostas_df = pd.DataFrame(respostas_df.tolist())
    # Garantir que todas as perguntas estejam como colunas
    for pergunta in perguntas:
        if pergunta not in respostas_df.columns:
            respostas_df[pergunta] = "Não sei"
    return respostas_df

In [11]:
conn = duckdb.connect("dados_namus.duckdb")

In [12]:
df = conn.execute("""
SELECT
	idFormatted,
	createdDateTime,
	modifiedDateTime,
	firstName,
	middleName,
	lastName,
	computedMissingMinAge,
	computedMissingMaxAge,
	circumstancesOfDisappearance,
	sex,
	hairColor,
	leftEyeColor,
	rightEyeColor,
	ethnicity,
	sightingDate,
	sightingCity,
	sightingState,
	sightingCounty,
	imageHref,
	sightingLat,
	sightingLon,
	primaryAgencyName,
	investigatingAgencyName,
	investigatingCaseNumber,
	caseDateReported,
	permissionToPublish,
	viewPermission,
	--hrefThumbnail,
	--hrefQRCode,
	caseIsResolved,
	vehicles
FROM
	dados_namus.main.MissingPersons;
""").df()

df['sightingLat'] = pd.to_numeric(df['sightingLat'], errors='coerce')
df['sightingLon'] = pd.to_numeric(df['sightingLon'], errors='coerce')
df

,idFormatted,createdDateTime,modifiedDateTime,firstName,middleName,lastName,computedMissingMinAge,computedMissingMaxAge,circumstancesOfDisappearance,sex,...,sightingLat,sightingLon,primaryAgencyName,investigatingAgencyName,investigatingCaseNumber,caseDateReported,permissionToPublish,viewPermission,caseIsResolved,vehicles
0,MP132669,2024-11-06T05:12:54.61,2024-11-07T21:44:23.637,Shayna,Michelle,Worthington,15.0,15.0,Unknown location,Female,...,28.765413,-82.453613,Citrus County Sheriff's Office,Citrus County Sheriff's Office,2024-9878,2009-11-06,False,public,False,[]
1,MP132709,2024-11-06T21:46:44.643,2024-11-08T16:30:06.637,BETTY,J,PELLUM,16.0,16.0,Unknown location,Female,...,30.277594,-81.723999,Jacksonville Sheriff's Office,Jacksonville Sheriff's Office,24-680281,2024-11-06,False,public,False,[]
2,MP132604,2024-11-05T14:18:31.87,2024-11-05T14:28:05.607,ALEKZANDER,FRANCIS JAMES,BURNETT,13.0,13.0,RUNAWAY,Male,...,30.210230,-81.745888,Jacksonville Sheriff's Office,Jacksonville Sheriff's Office,24-676929,2024-11-05,False,public,False,[]
3,MP132661,2024-11-06T02:45:27.397,2024-11-07T15:12:25,Alissa,N.,Plunkett,16.0,16.0,Last seen: wearing a brown & white short-sleev...,Female,...,30.464928,-84.385742,Leon County Sheriff's Office,Leon County Sheriff's Office,240157738,2024-11-05,False,public,False,[]
4,MP132656,2024-11-06T02:14:32.833,2024-11-07T15:09:22.313,Shannon,Ariannie,Salguero,16.0,16.0,Juvenile last seen Nov 05 2024,Female,...,26.142036,-81.794807,Collier County Sheriff's Office,Collier County Sheriff's Office,24-421449,2024-11-05,False,public,False,[]
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9741,MP94359,2022-08-15T18:32:29.437,2022-08-15T19:56:36.487,Carlos,Antonio,Gomez,1.0,1.0,On 3/14/2015 at about 3:00 pm The complainant ...,Male,...,32.862164,-96.872627,Dallas Police Department,Dallas Police Department,057832-2022,2022-08-15,False,public,False,[]
9742,MP38651,2017-06-15T13:22:00,2021-09-08T15:25:41.773,Joy,Melody,Mercado,15.0,15.0,"Joy was last seen on March 12, 2015.",Female,...,34.179985,-118.460121,Los Angeles Police Department,Los Angeles Police Department,150907500,2015-03-13,False,public,False,[]
9743,MP74205,2020-10-01T17:28:54.167,2023-10-05T18:41:44.877,Larry,None,Brown,55.0,55.0,Last seen in Staten Island.,Male,...,40.631145,-74.173904,New York City Police Department - Missing Pers...,New York City Police Department - Missing Pers...,KNMP2195,None,False,public,False,[]
9744,MP28794,2015-05-21T11:55:47,2024-05-21T12:14:02.47,Lee,Gregory,Snellings,23.0,23.0,None,Male,...,37.377094,-77.504990,Chesterfield County Police Department,Chesterfield County Police Department,201503290107,2015-03-29,False,public,False,"[{""id"": 4691, ""vehicleYear"": 2005, ""vehicleCol..."


In [13]:
df.to_csv('nome_do_arquivo.csv', index=False)

In [26]:
#  Definir constantes e estruturas de dados
perguntas = [
    "Nome?",
    "Sexo?",
    "Qual data de nascimento do desaparecido?",
    "Qual é altura aproximada do desaparecido?",
    "Qual é peso aproximada do desaparecido?",
    "Qual data do desaparecimento?",
    "Quanto tempo levou para ser encontrado?",
    "O desaparecido é homem?",
    "O desaparecido tem apelido?",
    "O desaparecido tem cabelo longo?",
    "O desaparecido tem olhos claros?",
    "O desaparecido tem problema de visão?",
    "O desaparecido usa óculos?",
    "O desaparecido tem alguma deficiências físicas?",
    "O desaparecido tem tatuagens?",
    "O desaparecido tem piercings?",
    "O desaparecido é Brasileiro?",
    "O desaparecido foi sequestro?",
    "O desaparecido tem relação com conflitos de guarda?",
    "O desaparecido se perdeu?",
    "O desaparecido tem amputações?",
    "O desaparecido estava procurando emprego?",
    "O desaparecido iria visitar parentes?",
    "O desaparecido mencionou a data de retornar?",
    "O desaparecido viajaria sozinho?",
    "O desaparecido iria encontrar alguém específico?",
    "O desaparecido já havia comentado sobre a possibilidade de deixar casa nunca mais voltar?",
    "O desaparecido já havia comentado sobre a possibilidade de deixar trabalho?",
    "O desaparecido possui algum objeto de grande valor sentimental em que foi levado com ele?",
    "O desaparecido mencionou planos ou ambições para o futuro recentemente?",
    "O desaparecido mantinha esperança apesar das dificuldades?",
    "O desaparecido iria para fora do Brasil?",
    "O desaparecido iria para Centro-Oeste do Brasil?",
    "O desaparecido iria para Nordeste do Brasil?",
    "O desaparecido iria para Norte do Brasil?",
    "O desaparecido iria para Sudeste do Brasil?",
    "O desaparecido iria para Sul do Brasil?",
    "O desaparecido retornava periodicamente à residência familiar?",
    "O desaparecido participava regularmente de encontros ou eventos comunitários?",
    "O desaparecido costumava usar carro?",
    "O desaparecido costumava usar ônibus?",
    "O desaparecido costumava usar aplicativo de transporte?",
    "O desaparecido viajava principalmente durante o dia?",
    "O desaparecido tem algum parente distante?",
    "O desaparecido é desempregado?",
    "O desaparecido possui características físicas asiáticas?",
    "O desaparecido possui características físicas africanas?",
    "O desaparecido possui características físicas europeias?",
    "O desaparecido possui características físicas latinas?",
    "O desaparecido possui características físicas americanas?",
    "O desaparecido se sentia mais feliz que a maioria das pessoas?",
    "Em geral, o desaparecido estava satisfeita com a sua vida?",
    "Em geral, as coisas estavam indo bem para o desaparecido?",
    "O desaparecido estava na posse de seus documentos pessoais?",
    "A pessoa desaparecida expressou recentemente o desejo de reencontrar um filho que foi doado?",
    "A pessoa desaparecida fez viagens recentes sem explicação clara?",
    "A pessoa desaparecida mencionou dificuldades em cuidar ou sustentar uma criança?",
    "O hospital forneceu uma explicação clara para o óbito do possível desaparecido?",
    "Você notou demora na comunicação do óbito do possível desaparecido por parte do hospital?",
    "O hospital facilitou o acesso a registros médicos relacionados ao caso do desaparecido?",
    "Alguém viu o corpo do desaparecido?",
    "O desaparecido está morto?"
]

# Definir perguntas que serão automaticamente preenchidas para crianças
perguntas_irrelevantes_para_criancas = [
    "O desaparecido costumava usar carro?",
    "O desaparecido costumava usar ônibus?",
    "O desaparecido costumava usar aplicativo de transporte?",
    "O desaparecido é desempregado?",
    "O desaparecido estava procurando emprego?",
    "O desaparecido já havia comentado sobre a possibilidade de deixar trabalho?",
    # Adicione mais perguntas conforme necessário
]

# Novas perguntas geológicas
perguntas_geologicas = [
    "A área apresenta rochas com camadas visíveis ou estratificadas?",
    "As rochas na área são duras e possuem superfícies brilhantes?",
    "As rochas têm uma aparência arenosa ou granular?",
    "Há presença de fósseis ou formas orgânicas nas rochas?",
    "Existe uma elevação alta como uma montanha ou colina próxima ao local?",
    "Há grandes falésias ou penhascos rochosos na área?",
    "Observa-se formações rochosas arqueadas ou colunas altas e finas?",
    "Há sinais de atividade vulcânica, como crateras ou fluxos de lava?",
    "Existem rios, lagos ou quedas d'água (cataratas) nas proximidades do local?",
    "Há evidências de geleiras, como depósitos de rochas ou vales profundos?",
    "São encontrados minerais metálicos visíveis, como ouro ou prata, na área?",
    "Há presença de gemas ou cristais brilhantes nas rochas?"
]

# Combinar as listas de perguntas
perguntas.extend(perguntas_geologicas)

# 2. Definir o mapeamento das perguntas para as colunas do DataFrame
mapeamento = {
    "Nome?": ["firstName", "middleName", "lastName"],
    "Sexo?": ["sex"],
    "Qual data de nascimento do desaparecido?": [],
    "Qual é altura aproximada do desaparecido?": [],
    "Qual é peso aproximada do desaparecido?": [],
    "Qual data do desaparecimento?": ["sightingDate"],
    "Quanto tempo levou para ser encontrado?": [],
    "O desaparecido é homem?": ["sex"],
    "O desaparecido tem apelido?": [],
    "O desaparecido tem cabelo longo?": [],
    "O desaparecido tem olhos claros?": ["leftEyeColor", "rightEyeColor"],
    "O desaparecido tem problema de visão?": [],
    "O desaparecido usa óculos?": [],
    "O desaparecido tem alguma deficiências físicas?": [],
    "O desaparecido tem tatuagens?": [],
    "O desaparecido tem piercings?": [],
    "O desaparecido é Brasileiro?": [],  # Será tratado pelas regras
    "O desaparecido foi sequestro?": [],
    "O desaparecido tem relação com conflitos de guarda?": [],
    "O desaparecido se perdeu?": [],
    "O desaparecido tem amputações?": [],
    "O desaparecido estava procurando emprego?": [],  # Será tratado pelas regras
    "O desaparecido iria visitar parentes?": [],
    "O desaparecido mencionou a data de retornar?": [],
    "O desaparecido viajaria sozinho?": [],
    "O desaparecido iria encontrar alguém específico?": [],
    "O desaparecido já havia comentado sobre a possibilidade de deixar casa nunca mais voltar?": [],
    "O desaparecido já havia comentado sobre a possibilidade de deixar trabalho?": [],  # Será tratado pelas regras
    "O desaparecido possui algum objeto de grande valor sentimental em que foi levado com ele?": [],
    "O desaparecido mencionou planos ou ambições para o futuro recentemente?": [],
    "O desaparecido mantinha esperança apesar das dificuldades?": [],
    "O desaparecido iria para fora do Brasil?": [],  # Será tratado pelas regras
    "O desaparecido iria para Centro-Oeste do Brasil?": [],
    "O desaparecido iria para Nordeste do Brasil?": [],
    "O desaparecido iria para Norte do Brasil?": [],
    "O desaparecido iria para Sudeste do Brasil?": [],
    "O desaparecido iria para Sul do Brasil?": [],
    "O desaparecido retornava periodicamente à residência familiar?": [],
    "O desaparecido participava regularmente de encontros ou eventos comunitários?": [],
    "O desaparecido costumava usar carro?": [],  # Será tratado pelas regras
    "O desaparecido costumava usar ônibus?": [],  # Será tratado pelas regras
    "O desaparecido costumava usar aplicativo de transporte?": [],  # Será tratado pelas regras
    "O desaparecido viajava principalmente durante o dia?": [],
    "O desaparecido tem algum parente distante?": [],
    "O desaparecido é desempregado?": [],  # Será tratado pelas regras
    "O desaparecido possui características físicas asiáticas?": ["ethnicity"],
    "O desaparecido possui características físicas africanas?": ["ethnicity"],
    "O desaparecido possui características físicas europeias?": ["ethnicity"],
    "O desaparecido possui características físicas latinas?": ["ethnicity"],
    "O desaparecido possui características físicas americanas?": ["ethnicity"],
    "O desaparecido se sentia mais feliz que a maioria das pessoas?": [],
    "Em geral, o desaparecido estava satisfeita com a sua vida?": [],
    "Em geral, as coisas estavam indo bem para o desaparecido?": [],
    "O desaparecido estava na posse de seus documentos pessoais?": [],
    "A pessoa desaparecida expressou recentemente o desejo de reencontrar um filho que foi doado?": [],
    "A pessoa desaparecida fez viagens recentes sem explicação clara?": [],
    "A pessoa desaparecida mencionou dificuldades em cuidar ou sustentar uma criança?": [],
    "O hospital forneceu uma explicação clara para o óbito do possível desaparecido?": [],
    "Você notou demora na comunicação do óbito do possível desaparecido por parte do hospital?": [],
    "O hospital facilitou o acesso a registros médicos relacionados ao caso do desaparecido?": [],
    "Alguém viu o corpo do desaparecido?": [],
    "O desaparecido está morto?": ["caseIsResolved"]
}

mapeamento_caracteristica_para_perguntas = {
    "Rochas sedimentares ou facies indiferenciadas": {
        "A área apresenta rochas com camadas visíveis ou estratificadas?": "Sim",
        "As rochas têm uma aparência arenosa ou granular?": "Sim",
        "Há presença de fósseis ou formas orgânicas nas rochas?": "Sim",
    },
    "Rochas vulcânicas extrusivas (excluindo traps)": {
        "Há sinais de atividade vulcânica, como crateras ou fluxos de lava?": "Sim",
        "Observa-se formações rochosas arqueadas ou colunas altas e finas?": "Sim",  # Ex.: Colunas de basalto
        "As rochas na área são duras e possuem superfícies brilhantes?": "Sim",  # Se houver vidro vulcânico
    },
    "Rochas endógenas (plutônicas e/ou metamórficas)": {
        "As rochas na área são duras e possuem superfícies brilhantes?": "Sim",
        "Existe uma elevação alta como uma montanha ou colina próxima ao local?": "Sim",
        "Há grandes falésias ou penhascos rochosos na área?": "Sim",
        "Há presença de gemas ou cristais brilhantes nas rochas?": "Sim",  # Minerais metamórficos
        "São encontrados minerais metálicos visíveis, como ouro ou prata, na área?": "Sim",
    },
    "Facies indiferenciadas": {
        "A área apresenta rochas com camadas visíveis ou estratificadas?": "Sim",
    },
    "Glacier inland ice": {
        "Há evidências de geleiras, como depósitos de rochas ou vales profundos?": "Sim",
    },
    "Rochas subglaciais (localizadas abaixo do nível do mar - calotas polares da Antártida e Groenlândia)": {
        "Há evidências de geleiras, como depósitos de rochas ou vales profundos?": "Sim",
    },
    "Extensão da ice shelf (Antártida)": {
        "Há evidências de geleiras, como depósitos de rochas ou vales profundos?": "Sim",
    },
    "Província Igneosa (LIP)": {
        "Há sinais de atividade vulcânica, como crateras ou fluxos de lava?": "Sim",
        "Existe uma elevação alta como uma montanha ou colina próxima ao local?": "Sim",  # Possíveis planaltos vulcânicos
        "As rochas na área são duras e possuem superfícies brilhantes?": "Sim",  # Rochas ígneas
    },
    "Complexo ofiolítico de idade Meso-Cenozoica": {
        "As rochas na área são duras e possuem superfícies brilhantes?": "Sim",
        "Há grandes falésias ou penhascos rochosos na área?": "Sim",
        "Há presença de gemas ou cristais brilhantes nas rochas?": "Sim",  # Minerais como olivina
    },
    "Plataforma continental (profundidade menor que 200 m), Exceto Antártida": {
        "A área apresenta rochas com camadas visíveis ou estratificadas?": "Sim",
        "Há presença de fósseis ou formas orgânicas nas rochas?": "Sim",  # Depósitos marinhos
    },
}

geological_data = {
    "Cor": ["f2ff59", "73f2a6", "e6bf4c", "bf8040", "ffb233", "ffb2b2", "ff99e6", "99e633", "66d933", "d98c33", "bf73ff", "ff738c", "ff9980", "e6a6ff", "ff8cb2", "ffb2d9", "ff73cc", "1ac1ff", "e6bf4c", "bf8040", "ffb266", "ffb2b2", "ff99e6", "d9d966", "99e633", "66d933", "ffbf40", "ff9980", "ffb2d9", "bf73ff", "f2ff59", "73f2a6", "e6bf4c", "bf8040", "ffb266", "ffb2b2", "ff99e6", "d9d966", "99e633", "66d933", "d98c33", "ffbf40", "f273a6", "ff9980", "ffb2d9", "ff73cc", "f259f8", "bf73ff", "c0bebd", "f259f8", "d9d9f2", "d9d9f2", "99e6e6", "fd2b1b", "26ff80", "fff2e6"],
    "Tipo de Crosta": ["Continental", "Continental", "Continental", "Continental", "Continental", "Continental", "Continental", "Continental", "Continental", "Continental", "Continental", "Continental", "Continental", "Continental", "Continental", "Continental", "Continental", "Continental", "Continental", "Continental", "Continental", "Continental", "Continental", "Continental", "Continental", "Continental", "Continental", "Continental", "Continental", "Continental", "Continental", "Continental", "Continental", "Continental", "Continental", "Continental", "Continental", "Continental", "Continental", "Continental", "Continental", "Continental", "Continental", "Continental", "Continental", "Continental", "Continental", "Continental", "Continental", "Continental", "Glacial", "Subglacial", "Extensão", "Igneosa", "Ophiolítica", "Continental ou Insular"],
    "Tipo de Rocha": ["Rochas sedimentares ou facies indiferenciadas", "Rochas sedimentares ou facies indiferenciadas", "Rochas sedimentares ou facies indiferenciadas", "Rochas sedimentares ou facies indiferenciadas", "Rochas sedimentares ou facies indiferenciadas", "Rochas sedimentares ou facies indiferenciadas", "Rochas sedimentares ou facies indiferenciadas", "Rochas sedimentares ou facies indiferenciadas", "Rochas sedimentares ou facies indiferenciadas", "Rochas sedimentares ou facies indiferenciadas", "Rochas sedimentares ou facies indiferenciadas", "Rochas sedimentares ou facies indiferenciadas", "Rochas sedimentares ou facies indiferenciadas", "Rochas sedimentares ou facies indiferenciadas", "Rochas sedimentares ou facies indiferenciadas", "Rochas sedimentares ou facies indiferenciadas", "Rochas sedimentares ou facies indiferenciadas", "Rochas vulcânicas extrusivas (excluindo traps)", "Rochas vulcânicas extrusivas (excluindo traps)", "Rochas vulcânicas extrusivas (excluindo traps)", "Rochas vulcânicas extrusivas (excluindo traps)", "Rochas vulcânicas extrusivas (excluindo traps)", "Rochas vulcânicas extrusivas (excluindo traps)", "Rochas vulcânicas extrusivas (excluindo traps)", "Rochas vulcânicas extrusivas (excluindo traps)", "Rochas vulcânicas extrusivas (excluindo traps)", "Rochas vulcânicas extrusivas (excluindo traps)", "Rochas vulcânicas extrusivas (excluindo traps)", "Rochas vulcânicas extrusivas (excluindo traps)", "Rochas vulcânicas extrusivas (excluindo traps)", "Rochas endógenas (plutônicas e/ou metamórficas)", "Rochas endógenas (plutônicas e/ou metamórficas)", "Rochas endógenas (plutônicas e/ou metamórficas)", "Rochas endógenas (plutônicas e/ou metamórficas)", "Rochas endógenas (plutônicas e/ou metamórficas)", "Rochas endógenas (plutônicas e/ou metamórficas)", "Rochas endógenas (plutônicas e/ou metamórficas)", "Rochas endógenas (plutônicas e/ou metamórficas)", "Rochas endógenas (plutônicas e/ou metamórficas)", "Rochas endógenas (plutônicas e/ou metamórficas)", "Rochas endógenas (plutônicas e/ou metamórficas)", "Rochas endógenas (plutônicas e/ou metamórficas)", "Rochas endógenas (plutônicas e/ou metamórficas)", "Rochas endógenas (plutônicas e/ou metamórficas)", "Rochas endógenas (plutônicas e/ou metamórficas)", "Rochas endógenas (plutônicas e/ou metamórficas)", "Rochas endógenas (plutônicas e/ou metamórficas)", "Rochas endógenas (plutônicas e/ou metamórficas)", "Rochas endógenas (plutônicas e/ou metamórficas)", "Facies indiferenciadas", "Glacier inland ice", "Rochas subglaciais (localizadas abaixo do nível do mar - calotas polares da Antártica e Groenlândia)", "Extensão da ice shelf (Antártica)", "Província Igneosa (LIP)", "Complexo ofiolítico de idade Meso-Cenozoica", "Plataforma continental (profundidade menor que 200 m), Exceto Antártida"],
    "Idade Geológica": ["Cenozoico", "Mesozoico", "Paleozóico Superior", "Paleozóico Inferior", "Neoproterozoico", "Mesoproterozoico", "Paleoproterozoico", "Mesozoico a Paleozóico Superior", "Mesozoico a Paleozóico", "Paleozóico", "Paleoproterozoico a Arqueano", "Paleozóico a Pré-Cambriano", "Neoproterozoico a Paleozóico Inferior", "Mesoproterozoico a Paleoproterozoico", "Neoproterozoico a Mesoproterozoico", "Proterozoico", "Pré-Cambriano", "Cenozoico", "Paleozóico Superior", "Paleozóico Inferior", "Neoproterozoico", "Mesoproterozoico", "Paleoproterozoico", "Cenozoico a Mesozoico", "Mesozoico a Paleozóico Superior", "Mesozoico a Paleozóico", "Paleozóico a Neoproterozoico", "Paleozóico Inferior a Neoproterozoico", "Proterozoico", "Paleoproterozoico a Arqueano", "Cenozoico", "Mesozoico", "Paleozóico Superior", "Paleozóico Inferior", "Neoproterozoico", "Mesoproterozoico", "Paleoproterozoico", "Cenozoico a Mesozoico", "Mesozoico a Paleozóico Superior", "Mesozoico a Paleozóico", "Paleozóico", "Paleozóico a Neoproterozoico", "Paleozóico a Paleoproterozoico", "Paleozóico Inferior a Neoproterozoico", "Proterozoico", "Pré-Cambriano", "Mesoproterozoico a Arqueano", "Paleoproterozoico a Arqueano", "Idade desconhecida", "Arqueano", "-", "-", "-", "-", "-", "-"]
}

In [27]:
mapeamento_subdivisoes = construir_mapeamento_subdivisoes()

In [28]:
geological = pd.DataFrame(geological_data)

# Preparar o dataframe geological
geological['Cor'] = geological['Cor'].str.lower()
geological['RGB'] = geological['Cor'].apply(hex_to_rgb)

# Inicializar o mapeador geológico
mapa_caminho = './mapa/mapa.tif'  # Atualize o caminho conforme necessário
geological_mapper = GeologicalMapper(mapa_caminho, geological)

In [29]:
# Atualizar o dataframe com a característica geológica
df['Caracteristica Geologica'] = df.apply(
    lambda row: geological_mapper.obter_caracteristica_geologica(row['sightingLat'], row['sightingLon']),
    axis=1
)

# Gerar o dataframe de respostas
respostas_df = gerar_respostas(df)
respostas_df

,A área apresenta rochas com camadas visíveis ou estratificadas?,As rochas na área são duras e possuem superfícies brilhantes?,As rochas têm uma aparência arenosa ou granular?,Há presença de fósseis ou formas orgânicas nas rochas?,Existe uma elevação alta como uma montanha ou colina próxima ao local?,Há grandes falésias ou penhascos rochosos na área?,Observa-se formações rochosas arqueadas ou colunas altas e finas?,"Há sinais de atividade vulcânica, como crateras ou fluxos de lava?","Existem rios, lagos ou quedas d'água (cataratas) nas proximidades do local?","Há evidências de geleiras, como depósitos de rochas ou vales profundos?",...,"Em geral, as coisas estavam indo bem para o desaparecido?",O desaparecido estava na posse de seus documentos pessoais?,A pessoa desaparecida expressou recentemente o desejo de reencontrar um filho que foi doado?,A pessoa desaparecida fez viagens recentes sem explicação clara?,A pessoa desaparecida mencionou dificuldades em cuidar ou sustentar uma criança?,O hospital forneceu uma explicação clara para o óbito do possível desaparecido?,Você notou demora na comunicação do óbito do possível desaparecido por parte do hospital?,O hospital facilitou o acesso a registros médicos relacionados ao caso do desaparecido?,Alguém viu o corpo do desaparecido?,O desaparecido está morto?
0,Não sei,Não sei,Não sei,Não sei,Não sei,Não sei,Não sei,Não sei,Não sei,Não sei,...,Não sei,Não sei,Não sei,Não sei,Não sei,Não sei,Não sei,Não sei,Não sei,Não
1,Não sei,Não sei,Não sei,Não sei,Não sei,Não sei,Não sei,Não sei,Não sei,Não sei,...,Não sei,Não sei,Não sei,Não sei,Não sei,Não sei,Não sei,Não sei,Não sei,Não
2,Não sei,Não sei,Não sei,Não sei,Não sei,Não sei,Não sei,Não sei,Não sei,Não sei,...,Não sei,Não sei,Não sei,Não sei,Não sei,Não sei,Não sei,Não sei,Não sei,Não
3,Não sei,Não sei,Não sei,Não sei,Não sei,Não sei,Não sei,Não sei,Não sei,Não sei,...,Não sei,Não sei,Não sei,Não sei,Não sei,Não sei,Não sei,Não sei,Não sei,Não
4,Não sei,Não sei,Não sei,Não sei,Não sei,Não sei,Não sei,Não sei,Não sei,Não sei,...,Não sei,Não sei,Não sei,Não sei,Não sei,Não sei,Não sei,Não sei,Não sei,Não
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9741,Não sei,Não sei,Não sei,Não sei,Não sei,Não sei,Não sei,Não sei,Não sei,Não sei,...,Não sei,Não sei,Não sei,Não sei,Não sei,Não sei,Não sei,Não sei,Não sei,Não
9742,Não sei,Não sei,Não sei,Não sei,Não sei,Não sei,Não sei,Não sei,Não sei,Não sei,...,Não sei,Não sei,Não sei,Não sei,Não sei,Não sei,Não sei,Não sei,Não sei,Não
9743,Não sei,Não sei,Não sei,Não sei,Não sei,Não sei,Não sei,Não sei,Não sei,Não sei,...,Não sei,Não sei,Não sei,Não sei,Não sei,Não sei,Não sei,Não sei,Não sei,Não
9744,Não sei,Não sei,Não sei,Não sei,Não sei,Não sei,Não sei,Não sei,Não sei,Não sei,...,Não sei,Não sei,Não sei,Não sei,Não sei,Não sei,Não sei,Não sei,Não sei,Não
